In [ ]:
import pandas as pd

# Read grades
df = pd.read_excel("Grade_CS_Students.xlsx", na_values=['NA'])

# Drop unnecessary columns
df = df.drop(['Year of enrolment', 'ID'], axis=1)

# Read titles of targets and features
target_dict = pd.read_excel("targets.xlsx").set_index('Code')['Desc'].to_dict()
features_dict = pd.read_excel("features.xlsx").set_index('Code')['Desc'].to_dict()

# Split data into target and features
features = df.drop(target_dict.keys(), axis=1)
targets = df[target_dict.keys()]

# Encode features into grades
def encode_grade(marks):
    # Convert scores into grades
    if marks > 85:
        return 0  #'A+'
    elif 80 <= marks <= 85:
        return 1  #'A'
    elif 75 <= marks < 80:
        return 2  #'A-'
    elif 70 <= marks < 75:
        return 3  #'B+'
    elif 65 <= marks < 70:
        return 4  #'B'
    elif 60 <= marks < 65:
        return 5  #'B-'
    elif 55 <= marks < 60:
        return 6  #'C+'
    elif 50 <= marks < 55:
        return 7  #'C'
    elif 45 <= marks < 50:
        return 8  #'C-'
    elif 40 <= marks < 45:
        return 9 #'D+'
    elif 35 <= marks < 40:
        return 10 #'D'
    elif marks < 35:
        return 11 #'E'
    else:
        return pd.NA
    
# Encode features and features
encoded_features = pd.DataFrame()
for column in features.columns:
    encoded_features[column] = features[column].apply(encode_grade)
encoded_targets = pd.DataFrame()
for column in targets.columns:
    encoded_targets[column] = targets[column].apply(encode_grade)
    
# Compare NaN values before and after    
nan_before = features.isna()
nan_after = encoded_features.isna()
if (nan_after.equals(nan_before) == False):
    print("WARNING: NaN mismatch in feature encoding")
nan_before = targets.isna()
nan_after = encoded_targets.isna()
if (nan_after.equals(nan_before) == False):
    print("WARNING: NaN mismatch in target encoding")

# Save encoded features and targets
encoded_features.to_csv('encoded_features.csv', index=False)    
encoded_targets.to_csv('encoded_targets.csv', index=False)

In [ ]:
# Set up matplotlib's interactive mode
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

encoded_features = pd.read_csv('encoded_features.csv')
encoded_targets = pd.read_csv('encoded_targets.csv')

df = pd.concat([encoded_features, encoded_targets], axis=0)

# Null count
df.isnull().sum()

# Box plots
plt.figure(figsize=(20, 10))
sns.boxplot(data=df)
plt.xticks(rotation=90)
plt.show()

imputed_features = pd.DataFrame()
imputed_targets  = pd.DataFrame()

# Since the data is skewed replace with the median
for col in encoded_features.columns:
    imputed_features[col] = pd.to_numeric(encoded_features[col], errors='coerce')  # Convert to numeric, set non-numeric to NaN
    imputed_features.fillna({col: encoded_features[col].median()}, inplace=True)   # Fill NaN with median of the column
    
for col in encoded_targets.columns:
    imputed_targets[col] = pd.to_numeric(encoded_targets[col], errors='coerce')  # Convert to numeric, set non-numeric to NaN
    imputed_targets.fillna({col: encoded_targets[col].median()}, inplace=True)   # Fill NaN with median of the column
    
print(imputed_features.isna().sum()) 
print(imputed_targets.isna().sum()) 
    
# Save imputed features and targets
imputed_features.to_csv('imputed_features.csv', index=False)    
imputed_targets.to_csv('imputed_targets.csv', index=False)

In [ ]:
# EDA
from ydata_profiling import ProfileReport

encoded_features = pd.read_csv('encoded_features.csv')
encoded_targets = pd.read_csv('encoded_targets.csv')

profile_df = pd.concat([encoded_features, encoded_targets], axis=1)

# Generate the report
profile = ProfileReport(profile_df, title='Profile Report', explorative=True)
profile.to_file("profiling-report.html")

In [ ]:
# Correlation

import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np

# Read dataset
imputed_features = pd.read_csv('imputed_features.csv')    
imputed_targets  = pd.read_csv('imputed_targets.csv')

# Read titles of targets and features
target_dict = pd.read_excel("targets.xlsx").set_index('Code')['Desc'].to_dict()
features_dict = pd.read_excel("features.xlsx").set_index('Code')['Desc'].to_dict()

correlations = {}

for column in imputed_targets.columns:
    correlation = imputed_features.corrwith(imputed_targets[column])
    correlation.sort_values(inplace=True, ascending=False)
    correlations[column] = correlation

# Display correlations as a formatted table
for target_code, correlation in correlations.items():
    target_title = target_dict.get(target_code, "Unknown Target")
    
    data = []
    for feature_code, corr_value in correlation.items():
        feature_title = features_dict.get(feature_code, "Unknown Feature")
        data.append([f"{feature_title} ({feature_code})", np.round(corr_value, decimals=2)])
    
    print(tabulate(data, headers=[f"{target_title} ({target_code})", ""], tablefmt='grid'))

In [ ]:
# Train Models
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import json 
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

# Function to merge small classes into the next larger class
def merge_small_classes(target_column, threshold=4):
    if isinstance(target_column, np.ndarray):
        target_column = pd.Series(target_column)
    value_counts = target_column.value_counts()
    small_classes = value_counts[value_counts < threshold].index
    for small_class in small_classes:
        # Find the next larger class
        larger_classes = value_counts[value_counts >= threshold].index
        if len(larger_classes) > 0:
            next_larger_class = larger_classes[0]
            target_column[target_column == small_class] = next_larger_class
        else:
            # If no larger class exists, keep the class as is
            continue
    return target_column

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def train_model_and_evaluate_for_each_target(features_dict, targets_dict):
    # Iterate over each resampled dataset
    for column, (features, target) in features_dict.items():
        
        # Split the data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
        
        # Fix class imbalances
        y_train = merge_small_classes(y_train)
        smote = SMOTE(k_neighbors = 3)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
        
        #X_train_resampled, y_train_resampled = X_train, y_train
        
        # Labels should start from 0, 1, 2, ... for XGBoost
        le = LabelEncoder()
        y_train_resampled = le.fit_transform(y_train_resampled)
        y_test = le.fit_transform(y_test)
        
        # Setup model and hyperparameter tuning
        #model = RandomForestClassifier()
        #model_name = "rf"
        #gs_space = {
        #    'max_depth': [3,5,7,10],
        #    'n_estimators': [100, 200, 300, 400, 500],
        #    'max_features': [10, 20, 30 , 40],
        #    'min_samples_leaf': [1, 2, 4]
        #}
        
        '''
        classes_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(train_df['class']),
                                             train_df['class']))

        weights = np.ones(y_train.shape[0], dtype = 'float')
        for i, val in enumerate(y_train):
            weights[i] = classes_weights[val-1]
        '''
        
        model = XGBClassifier()
        model_name = "xg"
        gs_space = {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [2, 3, 4],
            'learning_rate': [0.1, 0.01, 0.001]
        }
        
        
        #SVM
        #model = SVC()
        #gs_space = {
        #    'kernel': ['rbf'],
        #}
        #model_name = 'svc'

        #MLP
        # model = MLPClassifier()
        # model_name = 'MLP'

        scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
        grid = GridSearchCV(model, gs_space, scoring=scoring, refit="roc_auc", verbose = 2, return_train_score=True, n_jobs=-1)
        model_grid = grid.fit(X_train_resampled, y_train_resampled)

        best_results = {}
        for col in model_grid.cv_results_.keys():
            best_results[col] = model_grid.cv_results_[col][model_grid.best_index_]

        best_model = model_grid.best_estimator_
        
        # Test Set Evaluation
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Calculate the confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)

        best_results['test_accuracy']     = accuracy
        best_results['test_precision']    = precision
        best_results['test_f1']           = f1
        best_results['test_recall']       = recall
        best_results['confusion_matrix']  = conf_matrix
        
        print(best_results)
        
        with open(f'{model_name}_{column}.txt', 'w') as file:
            file.write(json.dumps(best_results, cls=NpEncoder))

        
# Read dataset
imputed_features = pd.read_csv('imputed_features.csv').astype(int)  
imputed_targets  = pd.read_csv('imputed_targets.csv').astype(int)

features_dict = {}
targets_dict = {}

for column in imputed_targets.columns:
    features_dict[column] = (imputed_features, imputed_targets[column])
    targets_dict[column]  = imputed_targets[column]

train_model_and_evaluate_for_each_target(features_dict, targets_dict)